In [24]:
import re
import regex  # the re.sub(r"\r" and others don't work otherwise
import datetime
def prnow():
    print(datetime.datetime.now())
prnow()


2025-01-21 16:34:10.419844


In [25]:
pattern = br'hello'
text = b'hello world'
# Replace 'hello' with 'hi' in the bytes string
result = re.sub(pattern, b'hi', text)
print(result)
bytes = b"abcdefg"
string = bytes.decode("utf-8", "ignore") 
print(string)


b'hi world'
abcdefg


In [12]:
#utf16_string = b"Hello \u202f World".encode(utf-16,errors='strict')
#utf16_string = ('Hello' + b'\u202f' + 'World').encode(utf-16,errors='strict')
utf16_string = ('Hello' + u'\u202f' + 'World')
print(f"{type(utf16_string)}")
# Decode from UTF-16 to Python's internal Unicode representation
#decoded_string = utf16_string.decode('utf-16', errors='ignore')
#AttributeError: 'str' object has no attribute 'decode'
# Encode to UTF-8
utf8_string = utf16_string.encode('utf-8',errors='ignore')
print(utf8_string) 

<class 'str'>
b'Hello\xe2\x80\xafWorld'


In [13]:
print(b'Hello')
u16 = u'\x61\u0000202f\x62'
#print(u16.decode('utf-16').encode('utf-8') ) #AttributeError: 'str' object has no attribute 'decode'
print(u16)
print('\u202f')
print(u16)


b'Hello'
a 202fb
 
a 202fb


In [16]:
# normally would want separate outfile, but was just trying 'rb+' - it overwrites
inoutfile = './testfile'
try:
    with open(inoutfile, 'rb+') as fin:
        #bytestr = fin.read(1040)  # or use seek()
        bytestr = fin.seek(1040)
        
        fin.write(b'\x93\x41\x20\x42\x20\x16\x43\x20\u202f\x20\x44\x20')  # or writeline??
        print(f"{fin.tell()=}")
    fin.close()
except Exception as e:
    print(f"{e=}")
# it works with syntax warnings and is now binary file
print("wrote read/write ", inoutfile)

fin.tell()=1057
wrote read/write  ./testfile


<>:8: SyntaxWarning: invalid escape sequence '\u'
<>:8: SyntaxWarning: invalid escape sequence '\u'
C:\Users\krista\AppData\Local\Temp\ipykernel_3848\2755928918.py:8: SyntaxWarning: invalid escape sequence '\u'
  fin.write(b'\x93\x41\x20\x42\x20\x16\x43\x20\u202f\x20\x44\x20')  # or writeline??


In [17]:
# This identifies the first bad byte
infile = './testfile'
def numofbadbyte():
    numslist = []
    try:
        with open(infile, 'r', encoding='utf-8') as fin:
            utf8str = fin.read()
    except UnicodeDecodeError as e:
        print('unicode decode utf-8 has failed\n', e)
        nums = re.findall(r'\d+' , str(e))
        numslist.append(nums)
        print(f"{nums=}")
    except Exception as e:
        print('other err for utf8\n', e)
    print(f"{len(numslist)=} {numslist=}")
numofbadbyte()


unicode decode utf-8 has failed
 'utf-8' codec can't decode byte 0x93 in position 1040: invalid start byte
nums=['8', '0', '93', '1040']
len(numslist)=1 numslist=[['8', '0', '93', '1040']]


In [26]:
# Write similar to findlinenbrforbadbyte but without line and byte numbers
def printasplaintext():
    lnnbr = 0
    limit = 300  # CHANGE THIS FOR LONGER FILES
    print(f"{limit=}")
    print("starting open 'rb'")
    try:
        print("trying open 'rb'")
        with open(infile, 'rb') as fin:
            line_count = sum(1 for line in fin)
            fin.seek(0,0)
            fin.close()
        with open(infile, 'rb') as fin:
            print(f"{line_count=}")
            with open(outfile, 'w+') as fout:
                while(lnnbr < limit) and (lnnbr <= line_count):
                    lnnbr += 1  # in this case, increment just for trying for safety
                    try:
                        ln = fin.readline()
                        if len(ln) == 0:
                            print(f"if len(ln) is 0, break, {lnnbr=}")
                            break
                        #print("trying rb readline")
                        fout.write(str(ln) + '\n')
                    except Exception as e:
                        print(e)
                        return
    except Exception as e:
        print(f"{e=}")
    fin.close()  # not needed, but just to remember
    fout.close()
    with open(outfile, "rb") as f:
        num_lines = sum(1 for _ in f)
        print(f"{num_lines=}")
        f.close()
infile = './testfile'
outfile = './plaintext'
printasplaintext()
print("wrote file ", outfile)


limit=300
starting open 'rb'
trying open 'rb'
line_count=262
if len(ln) is 0, break, lnnbr=263
num_lines=262
wrote file  ./plaintext


In [34]:
# Now, could I feed the assert byte numbers, numslines[] into it?
# or read the generated output file which has literal \xdd and \\udddd
import regex # the re.sub(r"\r" and others don't work otherwise
def fixnonreadables(infile):
    lnnbr = 0
    limit = 300  # CHANGE THIS FOR LONGER FILES
    print(f"{limit=}")
    unicodenewlines = u'[\u00a0\u1680\u180E\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200A\u200B\u202F\u205F\u3000\uFEFF]'
    print("starting open 'r' utf-8")
    try:
        print("trying open 'r' utf8")
        with open(infile, 'r', encoding='utf-8') as fin:
            line_count = sum(1 for line in fin)
            fin.seek(0,0)
            with open(outfile, 'w+') as fout:
                while(lnnbr < limit) & (lnnbr <= line_count):
                    lnnbr += 1  # in this case, increment just for trying for safety
                    try:
                        ln = fin.readline()
                        newln = str(ln)
                        newln = re.sub(r"^b'" , "" , newln)
                        newln = re.sub(r'^b"' , "" , newln)
                        newln = re.sub(r"\\n'$" , "" , newln)  # works
                        newln = re.sub(r'\\n"$' , '' , newln)  # does work
                        newln = re.sub(r"\\xe2\\x80\\99" , "'" , newln)
                        newln = re.sub(r"\\xe2\\x80\\x93" , "-" , newln)
                        newln = re.sub(r"\\x93", "-", newln)  # works
                        newln = re.sub(r"\\x[0-9a-fA-F]{2}" , "" , newln)  # works
                        newln = re.sub(r"\\\\u[0-9a-fA-F]{8}", "\n", newln)  #
                        newln = re.sub(r"\\\\u[0-9a-fA-F]{4}", "\n", newln)  # works
                        fout.write(newln + '\n')
                    except Exception as e:
                        print(e)
                        return
    except Exception as e:
        print(f"{e=}")
    fin.close()  # not needed, but just to remember
    fout.close()
infile = './plaintext'
outfile = './alltransfixed'
fixnonreadables(infile)
print("wrote file ", outfile)
# now run cleanpy.sh or .py equivalent

limit=300
starting open 'r' utf-8
trying open 'r' utf8
wrote file  ./alltransfixed


In [33]:
def cleanexcessblanklines():
    lnnbr = 0
    limit = 300  # CHANGE THIS FOR LONGER FILES
    print(f"{limit=}")
    ln = ''
    try:
        print("trying open 'r' utf8")
        with open(infile, 'r', encoding='utf-8') as fin:
            line_count = sum(1 for line in fin)
            fin.seek(0,0)
            with open(outfile, 'w+') as fout:
                while(lnnbr < limit) & (lnnbr <= line_count):
                    lnnbr += 1  # in this case, increment just for trying for safety
                    try:
                        ln = fin.readline()
                        newln = str(ln)
                        #newln = re.sub(r"^b'" , "" , newln)
                        #newln = re.sub(r"'$" , "" , newln)
                        newln = re.sub(r"^[\t \r\n]*$" , "" , newln)
                        
                        fout.write(newln)
                    except Exception as e:
                        print(e)
                        return
    except Exception as e:
        print(f"{e=}")
    fin.close()  # not needed, but just to remember
    fout.close()
infile = './alltransfixed'
outfile = './alltranscleaned'
cleanexcessblanklines()
print("wrote file ", outfile)


limit=300
trying open 'r' utf8
wrote file  ./alltranscleaned


In [1]:
# This prints the whole file with line numbers and byte numbers
# It has \n as a literal as well as \xdd and \udddd
def writelinenbrsandbytes():
    global writecnt
    writecnt = 0
    lnnbr = 0
    limit = 300
    print(f"{limit=}")
    bytenbr = 0
    oldbytenbr = 0
    print("starting open 'rb'")
    try:
        print("trying open 'rb'")
        with open(infile, 'rb') as fin:
            line_count = sum(1 for line in fin)
            print(f"{line_count=}")
            fin.close()
        with open(infile, 'rb') as fin:
            with open(outfile, 'w+') as fout:
                while(lnnbr < limit) and (lnnbr < line_count):
                    try:
                        #print("trying rb readline")
                        ln = fin.readline()
                        '''
                        if str(ln) == '':
                            print("hit if not ln")
                            break
                        ''';
                        lnnbr += 1  # in this case, set the actual line number to print
                        oldbytenbr = bytenbr
                        nextnbr = bytenbr + 1
                        bytenbr = bytenbr + len(ln)
                        outline = 'line ' + str(lnnbr) + ' byte ' + str(nextnbr) + ' ' + str(ln) + ' byte ' + str(bytenbr) + '\n'
                        if ("\\u" in outline):
                            print('Unreadable problem byte in line ', lnnbr)
                        if ("\\xe2\\80" in outline) & ("\\93" not in outline) & ("\\99" not in outline):
                            print('special character in line ', lnnbr)
                        fout.write(outline)
                        writecnt += 1
                    except Exception as e:
                        print(e)
                        return
    except Exception as e:
        print(f"{e=}")
    fin.close()  # not needed, but just to remember
    fout.close()
infile = './bytetest.txt'
outfile = './bytetestByteNumber.txt'
writelinenbrsandbytes()
print("wrote ", writecnt, "lines to ", outfile)


limit=300
starting open 'rb'
trying open 'rb'
line_count=4
wrote  4 lines to  ./bytetestByteNumber.txt
